In [83]:
import pandas as pd
import numpy as np
df = pd.read_csv('dataset2.csv')

print(df.head(), df.shape)


   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520 (1337, 7)


In [84]:
df_encoded = df.copy()

# Créez des colonnes pour les différentes catégories
df_encoded['anorexie_severe'] = np.where((df['bmi'].between(0, 15.99)), True, False)
df_encoded['anorexie_moderée'] = np.where((df['bmi'].between(16, 17)), True, False)
df_encoded['surpoids'] = np.where((df['bmi'].between(25, 29.99)), True, False)
df_encoded['obésité_modéré'] = np.where((df['bmi'].between(30, 34.9)), True, False)
df_encoded['obésité_severe'] = np.where((df['bmi'].between(35, 53)), True, False)



print(df_encoded)


      age     sex     bmi  children smoker     region      charges  \
0      19  female  27.900         0    yes  southwest  16884.92400   
1      18    male  33.770         1     no  southeast   1725.55230   
2      28    male  33.000         3     no  southeast   4449.46200   
3      33    male  22.705         0     no  northwest  21984.47061   
4      32    male  28.880         0     no  northwest   3866.85520   
...   ...     ...     ...       ...    ...        ...          ...   
1332   50    male  30.970         3     no  northwest  10600.54830   
1333   18  female  31.920         0     no  northeast   2205.98080   
1334   18  female  36.850         0     no  southeast   1629.83350   
1335   21  female  25.800         0     no  southwest   2007.94500   
1336   61  female  29.070         0    yes  northwest  29141.36030   

      anorexie_severe  anorexie_moderée  surpoids  obésité_modéré  \
0               False             False      True           False   
1               False

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import FunctionTransformer
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer


# separation des features et de la variable cible
X = df_encoded.drop('charges', axis=1)
y = df_encoded['charges']



print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# # division du dataset en ensembles d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(f''' verif du split 80 20 
# 80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
# 20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

# preprocessing avec standardisation et transformation des categories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'bmi', 'children']),
        ('cat', OneHotEncoder(), ['region', 'sex', 'smoker','obésité_severe','obésité_modéré','surpoids','anorexie_moderée','anorexie_severe']),
    ]
)


# preprocessing avec standardisation et transformation des categories
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), ['age', 'bmi', 'children']),
#         ('cat', OneHotEncoder(), ['region', 'sex', 'smoker'])
#     ]
# )

# preprocessing = make_column_transformer(
#     (OneHotEncoder(), ['bmi_category','region', 'sex', 'smoker']),
#     (StandardScaler(), ['age', 'children'])
# )










verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 11)
      Y (la variable cible) : (1337,)


In [86]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# ... (votre code existant)

# Modèle SGD Regressor
sgd_regressor_model = SGDRegressor(random_state=42)

# Pipeline pour le SGD Regressor
sgd_regressor_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', sgd_regressor_model)
])


# Entraînement du modèle SGD Regressor
sgd_regressor_pipeline.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
sgd_regressor_predictions = sgd_regressor_pipeline.predict(X_test)

# new_data = pd.DataFrame({
#     'age': [35, 40, 30, 60],  # Exemple d'âges
#     'sex': ['female', 'male', 'female', 'female'],
#     'bmi': [22.5, 30.0, 25.0, 47],
#     'children': [1, 2, 0, 5],
#     'smoker': ['no', 'yes', 'no', 'yes'],
#     'region': ['southwest', 'northeast', 'southeast', 'southeast']
# })



# # Faire des prédictions avec le modèle optimisé
# predictions = sgd_regressor_pipeline.predict(new_data)

# # Ajouter les prédictions à votre DataFrame d'origine
# new_data['predicted_charges'] = predictions

# # Afficher le DataFrame avec les prédictions
# print(new_data)

# score = sgd_regressor_pipeline.score(X_train, y_train)
# print(score)


In [87]:
# AVEC LE Modele de regression ElasticNet


from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

import matplotlib.pyplot as plt



# création du pipeline de prétraitement et du modèle elasticnet
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', ElasticNet())])


#test de différents hyperparam d'alpha entre ridge et lasso 
param_grid_elasticnet = {
    'regressor__alpha': [1.0, 2, 3, 4],
    'regressor__l1_ratio': [0.3, 0.5, 0.7, 1]
    }

### grid search
grid_elasticnet = GridSearchCV(pipeline_rf, param_grid_elasticnet, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_elasticnet.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_elasticnet.predict(X_test)
# print(y_pred)
score = grid_elasticnet.score(X_train, y_train)
print(score)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle lasso: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle: {rmse}')


# Récupérer les meilleurs hyperparamètres
best_params = grid_elasticnet.best_params_
print("Meilleurs hyperparamètres:", best_params)

# Récupérer le modèle avec les meilleurs hyperparamètres
best_model = grid_elasticnet.best_estimator_


# new_data = pd.DataFrame({
#     'age': [35, 40, 30, 60],  # Exemple d'âges
#     'sex': ['female', 'male', 'female', 'female'],
#     'bmi': [22.5, 30.0, 25.0, 47],
#     'children': [1, 2, 0, 5],
#     'smoker': ['no', 'yes', 'no', 'yes'],
#     'region': ['southwest', 'northeast', 'southeast', 'southeast']
# })


# # Faire des prédictions avec le modèle optimisé
# predictions = best_model.predict(new_data)

# # Ajouter les prédictions à votre DataFrame d'origine
# new_data['predicted_charges'] = predictions

# # Afficher le DataFrame avec les prédictions
# print(new_data)


# # feature_names = preprocessor.get_feature_names_out()
# # print(feature_names)



# 0.6120666150641507
# Mean Squared Error du modèle lasso: 60801052.43566085
# Coefficient of Determination R² du modèle lasso: 0.669120838254013
# Root Mean Squared Error (RMSE) du modèle lasso: 7797.502961567943
# Meilleurs hyperparamètres: {'regressor__max_depth': 20, 'regressor__n_estimators': 200}
#    age     sex   bmi  children smoker     region  predicted_charges
# 0   35  female  22.5         1     no  southwest        8359.378446
# 1   40    male  30.0         2    yes  northeast       30301.875435
# 2   30  female  25.0         0     no  southeast        8670.986965
# 3   60  female  47.0         5    yes  southeast       30971.347171


# 0.9665290950043812
# Mean Squared Error du modèle lasso: 21212643.285459656
# Coefficient of Determination R² du modèle lasso: 0.884560853019168
# Root Mean Squared Error (RMSE) du modèle lasso: 4605.7185417109085
# Meilleurs hyperparamètres: {'regressor__max_depth': 10, 'regressor__n_estimators': 300}
#    age     sex   bmi  children smoker     region  predicted_charges
# 0   35  female  22.5         1     no  southwest        9254.140953
# 1   40    male  30.0         2    yes  northeast       27878.313691
# 2   30  female  25.0         0     no  southeast        4828.929915
# 3   60  female  47.0         5    yes  southeast       48704.356252


/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/pipeline.py", line 756, in score
    Xt = transform.transform(Xt)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/utils/_set_output.py", line 157, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sk

0.4299674302679001
Mean Squared Error du modèle lasso: 73402583.67185716
Coefficient of Determination R² du modèle: 0.4934811151027304
Root Mean Squared Error (RMSE) du modèle: 8567.530780327384
Meilleurs hyperparamètres: {'regressor__alpha': 1.0, 'regressor__l1_ratio': 0.3}


/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/pipeline.py", line 756, in score
    Xt = transform.transform(Xt)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/utils/_set_output.py", line 157, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sk

In [88]:
# AVEC LE Modele de regression ElasticNet


from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

import matplotlib.pyplot as plt



# création du pipeline de prétraitement et du modèle elasticnet
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', ElasticNet())])


#test de différents hyperparam d'alpha ridge
param_grid_elasticnet = {
    'regressor__alpha': [1.0, 2, 3, 4],
    'regressor__l1_ratio': [0]
    }

### grid search
grid_elasticnet = GridSearchCV(pipeline_rf, param_grid_elasticnet, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_elasticnet.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_elasticnet.predict(X_test)
# print(y_pred)
score = grid_elasticnet.score(X_train, y_train)
print(score)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle lasso: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle: {rmse}')


# Récupérer les meilleurs hyperparamètres
best_params = grid_elasticnet.best_params_
print("Meilleurs hyperparamètres:", best_params)

# Récupérer le modèle avec les meilleurs hyperparamètres
best_model = grid_elasticnet.best_estimator_


# new_data = pd.DataFrame({
#     'age': [35, 40, 30, 60],  # Exemple d'âges
#     'sex': ['female', 'male', 'female', 'female'],
#     'bmi': [22.5, 30.0, 25.0, 47],
#     'children': [1, 2, 0, 5],
#     'smoker': ['no', 'yes', 'no', 'yes'],
#     'region': ['southwest', 'northeast', 'southeast', 'southeast']
# })


# # Faire des prédictions avec le modèle optimisé
# predictions = best_model.predict(new_data)

# # Ajouter les prédictions à votre DataFrame d'origine
# new_data['predicted_charges'] = predictions

# # Afficher le DataFrame avec les prédictions
# print(new_data)

/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.160e+10, tolerance: 1.315e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.167e+10, tolerance: 1.298e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one o

0.35704489015216057
Mean Squared Error du modèle lasso: 85114667.57783115
Coefficient of Determination R² du modèle: 0.41266118502509597
Root Mean Squared Error (RMSE) du modèle: 9225.761083934005
Meilleurs hyperparamètres: {'regressor__alpha': 1.0, 'regressor__l1_ratio': 0}


In [91]:
# AVEC LE Modele de regression ElasticNet


from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

import matplotlib.pyplot as plt



# création du pipeline de prétraitement et du modèle elasticnet
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', ElasticNet())])


#test de différents hyperparam d'alpha lasso
param_grid_elasticnet = {
    'regressor__alpha': [1.0, 2, 3, 4, 5, 6],
    'regressor__l1_ratio': [1]
    }

### grid search
grid_elasticnet = GridSearchCV(pipeline_rf, param_grid_elasticnet, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_elasticnet.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_elasticnet.predict(X_test)
# print(y_pred)
score = grid_elasticnet.score(X_train, y_train)
print(score)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle lasso: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle: {rmse}')


# Récupérer les meilleurs hyperparamètres
best_params = grid_elasticnet.best_params_
print("Meilleurs hyperparamètres:", best_params)

# Récupérer le modèle avec les meilleurs hyperparamètres
best_model = grid_elasticnet.best_estimator_


# new_data = pd.DataFrame({
#     'age': [35, 40, 30, 60],  # Exemple d'âges
#     'sex': ['female', 'male', 'female', 'female'],
#     'bmi': [22.5, 30.0, 25.0, 47],
#     'children': [1, 2, 0, 5],
#     'smoker': ['no', 'yes', 'no', 'yes'],
#     'region': ['southwest', 'northeast', 'southeast', 'southeast']
# })


# # Faire des prédictions avec le modèle optimisé
# predictions = best_model.predict(new_data)

# # Ajouter les prédictions à votre DataFrame d'origine
# new_data['predicted_charges'] = predictions

# # Afficher le DataFrame avec les prédictions
# print(new_data)

/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/pipeline.py", line 756, in score
    Xt = transform.transform(Xt)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sklearn/utils/_set_output.py", line 157, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/home/utilisateur/Bureau/ML lib/ml/lib/python3.10/site-packages/sk

0.7432247902368283
Mean Squared Error du modèle lasso: 25286199.947334982
Coefficient of Determination R² du modèle: 0.8255110765872928
Root Mean Squared Error (RMSE) du modèle: 5028.53854985074
Meilleurs hyperparamètres: {'regressor__alpha': 1.0, 'regressor__l1_ratio': 1}
